# 사용 토큰 확인

In [102]:
import requests

url = "http://localhost:3000/api/get_limit"
response = requests.get(url, headers={'Content-Type': 'application/json'})
print(response.status_code)
print(response.json())


200
{'credits_left': 20, 'period': None, 'monthly_limit': 50, 'monthly_usage': 30}


# 관련 API 및 파이프라인 데모

In [110]:
from flask import Flask, request, jsonify
from openai import OpenAI
import os

app = Flask(__name__)
api_key = os.getenv('OPENAI_API_KEY')
#openai.api_key = api_key

client = OpenAI(
    api_key= api_key,
)


def get_audio_information(audio_ids):
    url = f"{base_url}/api/get?ids={audio_ids}"
    response = requests.get(url)
    return response.json()

def send_post_request(url, payload):
    url = "http://localhost:3000/api/custom_generate"
    headers = {'Content-Type': 'application/json'}
    response = requests.post(url, data=json.dumps(payload), headers=headers)
    return response.json()

@app.route('/generate_lyrics', methods=['POST'])
def generate_lyrics(data):
    
    #data = request.get_json()
    
    #genre = data.get('genre')
    #style = data.get('style')
    
    genre = data['genre']
    style = data['style']
    
    #if not genre or not style:
    #    return jsonify({'error': 'Genre and style are required'}), 400

    prompt = f"""
    
    너는 노래 가사를 작성할 거야. 
    설명 문구는 빼고 가사만 말해줘.
    노래의 장르와 스타일은 다음과 같아
    한국 정서에 맞는 좋은 퀄리티의 가사를 뽑아줘. :
    
    장르: {genre}
    스타일: {style}
    
    이 정보를 바탕으로, {genre} 장르와 {style} 스타일에 맞는 한국어 노래 가사를 작성해줘. 노래 가사는 다음과 같은 구조를 가질 수 있어:
    - 첫 번째 구절 (Verse 1)
    - 두 번째 구절 (Verse 2)
    - 첫 번째 구절 (Verse 1)
    - 두 번째 구절 (Verse 2)
    - 다리 (Bridge) (선택 사항)
    - 후렴구 (Chorus)
    
    가사는 최소한 200자 이상으로 작성해줘. 
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates Korean song lyrics based on the given genre and style."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500
    )

    return response.choices[0].message.content.strip()

class Translate_Model():

    def __init__(self):
        pass
        
    def __call__(self,text,input_language,output_language):

        change_code = {"ko":"KO","en":"EN"}
        params = {'auth_key' : '', 'text' : text, 'source_lang' : change_code[input_language], "target_lang": change_code[output_language] }
        result = requests.post('https://api-free.deepl.com/v2/translate', data=params, verify=False)
        print(result.json()['translations'][0]["text"])
            
        return result.json()['translations'][0]["text"]


In [112]:
translate = Translate_Model()

style = "출근중인데 퇴근하고 싶어, 발라드 스타일"
data = { "genre": 
    "발라드",
    "style": style
}
lyrcis = generate_lyrics(data)

style_default = "Good Quality, Korean,"
sex = "male,"
style_en = translate(style,"ko","en")
title = "퇴근하고 싶어"
title = translate(title,"ko","en")
tags = style_default + sex + style_en

print(title)
print(lyrcis)
print(tags)

/home/tester01/anaconda3/envs/mintai_llm/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api-free.deepl.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


I'm at work and I want to go home, ballad style


/home/tester01/anaconda3/envs/mintai_llm/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api-free.deepl.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


I want to leave work
I want to leave work
Verse 1:
출근길 지친 몸과 마음
희미한 불빛이 비쳐
가만히 멈춰 서있는
긴 하루 끝엔 또 어찌

Verse 2:
창문 넘어 비치는 빛이
내 맘을 비추고 있어
지친 하루가 끝나면
너에게로 달려가

Chorus:
퇴근하고 싶어
나의 모든 일상을 뒤로하고
풍경이 바뀌는 그 때
편히 눈 감고 싶어

Verse 1:
세상 속에 빠져있는
나를 찾아 안아주던
하루 끝에 웃음 짓는
너의 모습이 자꾸 보여

Verse 2:
조금씩 서서히 쌓인
일상 속에 갇힌 나
너와 함께 떠나고파
퇴근하고 싶어

Chorus:
퇴근하고 싶어
나의 모든 일상을 뒤로하고
풍경이 바뀌는 그 때
편히 눈 감고 싶어

(Bridge):
하고픈 게 많은데
일상에 얽혀버린 나인데
너와 함께라면
행복한 퇴근을 꿈꿀 수 있어

Chorus:
퇴근하고 싶어
나의 모든 일상을 뒤로하고
풍경이 바뀌는 그 때
편히 눈 감고 싶어
Good Quality, Korean,male,I'm at work and I want to go home, ballad style


In [113]:
import requests
import json

payload = {
    "prompt": f"{lyrcis}",
    "tags": f"{tags}",
    "title": f"{title}",
    "make_instrumental": False,
    "model": "chirp-v3-5",
    "wait_audio": False
}

response = send_post_request(url, payload)
print(response)


[{'id': '733db7ae-69bd-4c45-8216-2bf6075855ba', 'title': 'I want to leave work', 'image_url': None, 'lyric': 'Verse 1:\n출근길 지친 몸과 마음\n희미한 불빛이 비쳐\n가만히 멈춰 서있는\n긴 하루 끝엔 또 어찌\n\nVerse 2:\n창문 넘어 비치는 빛이\n내 맘을 비추고 있어\n지친 하루가 끝나면\n너에게로 달려가\n\nChorus:\n퇴근하고 싶어\n나의 모든 일상을 뒤로하고\n풍경이 바뀌는 그 때\n편히 눈 감고 싶어\n\nVerse 1:\n세상 속에 빠져있는\n나를 찾아 안아주던\n하루 끝에 웃음 짓는\n너의 모습이 자꾸 보여\n\nVerse 2:\n조금씩 서서히 쌓인\n일상 속에 갇힌 나\n너와 함께 떠나고파\n퇴근하고 싶어\n\nChorus:\n퇴근하고 싶어\n나의 모든 일상을 뒤로하고\n풍경이 바뀌는 그 때\n편히 눈 감고 싶어\n\n(Bridge):\n하고픈 게 많은데\n일상에 얽혀버린 나인데\n너와 함께라면\n행복한 퇴근을 꿈꿀 수 있어\n\nChorus:\n퇴근하고 싶어\n나의 모든 일상을 뒤로하고\n풍경이 바뀌는 그 때\n편히 눈 감고 싶어', 'audio_url': '', 'video_url': '', 'created_at': '2024-07-30T10:16:30.336Z', 'model_name': 'chirp-v3', 'status': 'submitted', 'gpt_description_prompt': None, 'prompt': 'Verse 1:\n출근길 지친 몸과 마음\n희미한 불빛이 비쳐\n가만히 멈춰 서있는\n긴 하루 끝엔 또 어찌\n\nVerse 2:\n창문 넘어 비치는 빛이\n내 맘을 비추고 있어\n지친 하루가 끝나면\n너에게로 달려가\n\nChorus:\n퇴근하고 싶어\n나의 모든 일상을 뒤로하고\n풍경이 바뀌는 그 때\n편히 눈 감고 싶어\n\nVerse 1:\n세상 속에 빠져있는\n나를 찾아 안아주던\n하루 끝에 웃음 짓는

In [114]:
data = response
ids = f"{data[0]['id']},{data[1]['id']}"
print(f"ids: {ids}")
data = get_audio_information(ids)
print(data)

ids: 733db7ae-69bd-4c45-8216-2bf6075855ba,1f2b2827-788d-4c03-993b-5976bc15fdd0


In [ ]:
import time
import requests

# replace your vercel domain
base_url = 'http://localhost:3000'


def custom_generate_audio(payload):
    url = f"{base_url}/api/custom_generate"
    response = requests.post(url, json=payload, headers={'Content-Type': 'application/json'})
    return response.json()


def extend_audio(payload):
    url = f"{base_url}/api/extend_audio"
    response = requests.post(url, json=payload, headers={'Content-Type': 'application/json'})
    return response.json()

def generate_audio_by_prompt(payload):
    url = f"{base_url}/api/generate"
    response = requests.post(url, json=payload, headers={'Content-Type': 'application/json'})
    return response.json()


def get_audio_information(audio_ids):
    url = f"{base_url}/api/get?ids={audio_ids}"
    response = requests.get(url)
    return response.json()


def get_quota_information():
    url = f"{base_url}/api/get_limit"
    response = requests.get(url)
    return response.json()

def get_clip(clip_id):
    url = f"{base_url}/api/clip?id={clip_id}"
    response = requests.get(url)
    return response.json()

def generate_whole_song(clip_id):
    payloyd = {"clip_id": clip_id}
    url = f"{base_url}/api/concat"
    response = requests.post(url, json=payload)
    return response.json()


if __name__ == '__main__':
    data = generate_audio_by_prompt({
        "prompt": """A popular heavy metal song about war, sung by a deep-voiced male singer, slowly and melodiously. 
        The lyrics depict the sorrow of people after the war.
        
        """,
        "make_instrumental": False,
        "wait_audio": False
    })

    ids = f"{data[0]['id']},{data[1]['id']}"
    print(f"ids: {ids}")

    for _ in range(60):
        data = get_audio_information(ids)
        if data[0]["status"] == 'streaming':
            print(f"{data[0]['id']} ==> {data[0]['audio_url']}")
            print(f"{data[1]['id']} ==> {data[1]['audio_url']}")
            break
        # sleep 5s
        time.sleep(5)
